# Специальные методы классов

In [1]:
import random

class Vector:        
    def __init__(self, x=0, y=0, color=None):
        print("initializing a vector")
        if type(x) != int or type(y) != int:
            raise AttributeError('x and y should be int')
        
        self._x = x
        self._y = y
        self._color = color
    
    def get_x(self):
        return self._x
    
    def get_y(self):
        return self._y

Методы с двойным подчеркиванием в начале и конце имени имеют особое значение. 

Мы уже знакомы с `__next__` и `__iter__`, пора узнать и об остальных

In [2]:
vector = Vector(1, 2, 'red')
str(vector)

initializing a vector


'<__main__.Vector object at 0x7f0d887064e0>'

In [3]:
class VectorWithStr(Vector):
    def __str__(self):
        return 'vector ({}, {}) of color {}'.format(self._x, self._y, self._color)

In [4]:
vector = VectorWithStr(1, 2, 'red')
str(vector)

initializing a vector


'vector (1, 2) of color red'

__Q:__ Преобразование в строку? Это всё?

__A:__ Конечно, нет. Неявные преобразования иногда происходят там, где мы их не ожидаем

In [5]:
print(vector)

vector (1, 2) of color red


In [6]:
print("OBJECT: {}".format(vector))

OBJECT: vector (1, 2) of color red


In [7]:
mydict = {}
mydict[vector]

KeyError: <__main__.VectorWithStr object at 0x7f0d887062e8>

In [9]:
mylist = [vector]
print(mylist)

__Q:__ А откуда опять "некрасивые" строки?!

__A:__ В питоне используется два способа приведения к строке. Это функции `str` и `repr`, которые отличаются своим назначением. 

`str` используется там, где нужна человекочитаемость, а `repr` реализуется так, чтобы можно было однозначно определить, о каком объекте идет речь. Если `repr` не реализован, используется стандартный вариант, а если не реализован `str`, то вместо него используется `repr`. 

In [10]:
class VectorWithRepr(Vector):
    def __repr__(self):
        return 'vector representation (x: {}, y: {}, color: {})'.format(self._x, self._y, self._color)

In [11]:
vector = VectorWithRepr(1, 2, 'red')

print(vector)
mylist = [vector]
print(mylist)
mydict = {}
mydict[vector] = 1

initializing a vector
vector representation (x: 1, y: 2, color: red)
[vector representation (x: 1, y: 2, color: red)]


In [100]:
class VectorWithBothReprAndStr(VectorWithRepr, VectorWithStr):
    pass

In [101]:
vector = VectorWithBothReprAndStr(1, 2, 'red')
# вот здесь должны получиться разные значения
print(vector)
print([vector])

initializing a vector
vector (1, 2) of color red
[vector representation (x: 1, y: 2, color: red)]


## Арифметика

In [102]:
import math
import random

class VectorWithMath(VectorWithBothReprAndStr):    
    def __abs__(self):
        return math.hypot(self._x, self._y)
    
    def __add__(self, other):
        return VectorWithMath(self.get_x() + other.get_x(),
                     self.get_y() + other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    def __sub__(self, other):
        return VectorWithMath(self.get_x() - other.get_x(),
                     self.get_y() - other.get_y(),
                     random.choice((str(self._color), str(other._color))))
    
    # ещё есть div, mul и многое другое

In [103]:
vector1 = VectorWithMath(3, 4, 'blue')
vector2 = VectorWithMath(1, 2, 'red')

initializing a vector
initializing a vector


In [104]:
print(abs(vector1))
print(vector1 + vector2)

5.0
initializing a vector
vector (4, 6) of color blue


## Приведение типов

In [201]:
import math

class VectorWithTypes(VectorWithMath):
    def __bool__(self):
        return bool(self._x) or bool(self._y)
    
    def __int__(self):
        return int(float(self))
    
    def __float__(self):
        return abs(self)

In [202]:
vector = VectorWithTypes(3, 4, 'blue')
print(vector)
print(int(vector))
print(float(vector))
if vector:
    print("vector ~ True")

initializing a vector
vector (3, 4) of color blue
5
5.0
vector ~ True


In [203]:
vector = VectorWithTypes()
print(vector)
if not vector:
    print("vector ~ False")

initializing a vector
vector (0, 0) of color None
vector ~ False


## Итерирование

Один способ сделать объект "итерабельным" нам уже известен, это метод `__next__`. Но он не единственный

In [204]:
class VectorIterable(VectorWithTypes):
    def __getitem__(self, position):
        return (self._x, self._y)[position]
    
    def __len__(self):
        return 2
    
    def __reversed__(self):
        return (self._x, self._y)[::-1]

In [205]:
vector = VectorIterable(100, 500)
vector[0]
vector[3]

initializing a vector


IndexError: tuple index out of range

In [206]:
for coordinate in vector:
    print(coordinate)

100
500


In [207]:
for coordinate in reversed(vector):
    print(coordinate)

500
100


## Динамическая работа с атрибутами

Казалось бы, в питоне нет никакой защиты от "взлома". Но нельзя ли сделать её самостоятельно?

In [208]:
class VectorWithAllAttributes(VectorIterable):
    def __getattr__(self, attr_name):
        return "value of {}".format(attr_name)
    
    def __setattr__(self, attr_name, attr_value):
        if attr_name not in ('_x', '_y', '_color'):
            raise Exception('you shall not add new attributes here, young padawan!')
        else:
            super().__setattr__(attr_name, attr_value)
            
    def __delattr__(self, attr_name):
        print('Heh, you can delete nothing')

In [211]:
vector = VectorWithAllAttributes(1, 2, 'violet')
print(dir(vector))

initializing a vector
['__abs__', '__add__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__float__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__int__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_color', '_x', '_y', 'get_x', 'get_y']


In [212]:
print(vector.some_attribute)
print(vector._color)
print(vector.get_x())

value of some_attribute
violet
1


In [213]:
vector.new_attribute = "value"

Exception: you shall not add new attributes here, young padawan!

In [214]:
delattr(vector, '_color')
print(vector._color)

Heh, you can delete nothing
violet


## Контексты

In [215]:
class VectorWithContextManager(VectorWithAllAttributes):
    def __enter__(self):
        print('entering context')
    def __exit__(self, *args):
        print(args)
        print('leaving context')

In [216]:
try:
    with VectorWithContextManager() as vec:
        for i in range(3):
            print(i)
        raise Exception('something happened inside!')
except Exception:
    print('an exception was raised...')
    pass
print('we are out if the context')

initializing a vector
entering context
0
1
2
(<class 'Exception'>, Exception('something happened inside!',), <traceback object at 0x7f9d886b0a08>)
leaving context
an exception was raised...
we are out if the context


Но можно и проще!

In [217]:
from contextlib import contextmanager

@contextmanager
def vector_mgr():
    print('handling entering the context')
    yield Vector()
    print('handling leaving the context')
          
print('statement before context')
with vector_mgr() as vector:
    print(vector)
print('statement after context')

statement before context
handling entering the context
initializing a vector
handling leaving the context
statement after context


## Создание и удаление объектов

In [240]:
class VectorInitialized(VectorWithContextManager):
    def __new__(cls, *args, **kwargs):
        print('invoking __new__ method')
        print(cls, args, kwargs)
        return object.__new__(cls)
    
    def __del__(self):
        print('deleting an object')
        raise Exception("exception while destructing")

In [241]:
vector = VectorInitialized(1, 2, color='navy blue')

invoking __new__ method
<class '__main__.VectorInitialized'> (1, 2) {'color': 'navy blue'}
initializing a vector


In [242]:
del vector

deleting an object


Exception ignored in: <bound method VectorInitialized.__del__ of vector representation (x: 1, y: 2, color: navy blue)>
Traceback (most recent call last):
  File "<ipython-input-240-b22a11d4ad3e>", line 9, in __del__
Exception: exception while destructing


### Упражнение! 

Как с помощью метода `__new__` сделать класс "синглтоном" -- объектом, который создается один раз, а при попытке повторного создания возвращается уже готовый объект?

In [ ]:
class SingletonClass:
    ...your code here...
    def __new__(cls, *args, **kwargs):
        ...your code here...

obj1 = SingletonClass()
obj2 = SingletonClass()
assert id(obj1) == id(obj2)